In [ ]:
import time
start_time = time.time()
import pandas as pd
import numpy as np
from scipy import *
from numpy import dot, multiply, diag, power
from numpy import pi, exp, sin, cos, cosh, tanh, real, imag
from numpy.linalg import inv, eig, pinv,norm
from scipy.linalg import svd, svdvals
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from io import BytesIO
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML
from __future__ import print_function
from sklearn import preprocessing  
from sklearn import metrics 
import scipy.io as sio
from xlrd import open_workbook 
import time  
import re
def save_npy_data():
    rootpath = 'E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/sub_features/one_sec'
    names = 'noise_level_'
    for i in range(1,11): 
        dataname = names + str(10*i)
        pathname = os.path.join(rootpath,dataname)
        data = sio.loadmat(pathname)
        testX_pred =  data['testX_pred_noise']
        testX_pred = np.reshape(testX_pred,(322,9,3,1))
        Energy_pred = data['Energy_pred_noise']
        Energy_pred = np.reshape(Energy_pred,(322,9,2,1))
        print (np.shape(testX_pred))
        print (np.shape(Energy_pred))
        noisename = names + str(10*i)
        savepath = os.path.join(rootpath, dataname+'/testX_pred.npy')
        np.save(savepath,testX_pred)
        Esavepath = os.path.join(rootpath, dataname+'/Energy_pred.npy')
        np.save(Esavepath,Energy_pred)

In [14]:
import os
import sys 
import numpy as np 
import tensorflow as tf
import operator
batch_size =50 
level =8

def load_model(session, save_path):
    """ Loads a saved TF model from a file.Returns:The inputs placehoder and the prediction operation. """
    print("Loading model from file '%s'..." % (save_path))
    meta_file = save_path + ".meta"
    if not os.path.exists(meta_file):
        print("ERROR: Expected .meta file '%s', but could not find it." % \
        (meta_file))
        sys.exit(1) 
    saver = tf.train.import_meta_graph(meta_file)
    save_path = os.path.join("./", save_path) 
    saver.restore(session, save_path)  
    return extract_validation_handles(session)
    
def load_validation_data(level=0):
    global batch_size 
    """ Loads the validation data,  Returns: A tuple of the loaded validation data and validation labels. """
    print("Loading validation data...")
    rootfolder = 'sub_features/one_sec' 
    all_X = []
    all_Y = []
    all_E = []
    for i in range(1,11):
        folder = 'noise_level_' + str(10*i)
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/sub_features/one_sec/'+folder+'/testX_pred.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/sub_features/one_sec/testY_pred.npy'  ) 
        Energy =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/sub_features/one_sec/'+folder+'/Energy_pred.npy' )
        #print (np.shape(testX))
        all_X.append(testX)
        all_Y.append(testY)
        all_E.append(Energy) 
    return (all_X[level], all_Y[level], all_E[level])  

def validate_model(session, val_data, x,y, e, phase_train,prob): 
    """ Validates the model stored in a session.Returns:The overall validation accuracy for the model. """
    global batch_size
    print("Validating model...")
    #y = tf.placeholder(tf.int32, [None, 3]) 
    predict_op = tf.argmax(prob,1)
    correct = tf.equal(predict_op,tf.argmax(y, 1))
    acc= tf.reduce_mean(tf.cast(correct, tf.float32))  
    val_x, val_y, val_e = val_data
    test_correct = session.run(correct,feed_dict={x: val_x, e:val_e,y :val_y , phase_train : True  })
    test_acc = session.run(acc,feed_dict={x: val_x, e:val_e,y :val_y , phase_train : True  })
    return ( test_acc), test_correct  
 
def extract_validation_handles(session):
    """ Extracts the input and predict_op handles that we use for validation.
    Args:
      session: The session with the loaded graph.
    Returns:
      The inputs placeholder, mask placeholder, and the prediction operation. """ 
    valid_nodes = tf.get_collection_ref("validation_nodes") 
    x = valid_nodes[0] 
    y = valid_nodes[1]  
    e = valid_nodes[2]  
    phase_train =valid_nodes[3]
    prob = valid_nodes[4]
    global batch_size 
    #batch_size = int(x.get_shape()[0])  
    # Predict op should also yield integers.
    #predict = tf.cast(predict_op, "int32") 
    # Check the shape of the prediction output.
    #p_shape = predict.get_shape() 
    return (x,y,e, phase_train,prob)

def each_perform(correct_results,eval_labels ):
    label_y=np.argmax(eval_labels,1) 
    
    acc = []
    zero = np.where(label_y==0) 
    correctl=[correct_results[i] for i in zero]
    accuracy_zero= np.mean(correctl,dtype=np.float32)
    acc.append(accuracy_zero)

    one = np.where(label_y==1)
    correctg=[correct_results[i] for i in one] 
    accuracy_one= np.mean(correctg,dtype=np.float32)
    acc.append(accuracy_one)

    two = np.where(label_y==2)
    correctt=[correct_results[i] for i in two] 
    accuracy_two= np.mean(correctt,dtype=np.float32)
    acc.append(accuracy_two) 
    #print('The accuracy of the linetrip events is %.2f \n'  %(100*accuracy_zero))
    #print('The accuracy of the generator trip events is %.2f \n' %(100-100*accuracy_one))
    #print('The accuracy of the line faults events is %.2f \n' %(100-100*accuracy_two) )
    return acc, correctl,correctg,correctt
def reset():
    tf.reset_default_graph() 
level = 5 # level i denotes snr = i+1
model_name ='modelA_snr'+str((level)*10)+'_fully'
used = [768, 4, 261, 262, 11, 12, 791, 26, 177, 951, 957, 192, 964, 716, 717, 980, 985, 219, 733, 227, 229, 741, 1127, 250, 1135, 752, 753, 1139, 1140, 756, 757, 246, 1146, 1149]
 
with tf.Session() as session:   
    best_model = 'E:/05_research/04_deep_learning_model/Save_parameters/modelA/Best_noise_model/'+ model_name #os.path.join(save_path,model_name) 
    x,y,e,phase_train,prob = load_model(session, best_model)
    val_data = load_validation_data(level-1) # the predicted mode
    total = [i for i in range(np.shape(val_data[0])[0])]
    remain = list(set(total) - set(used)); 
    accuracy ,correct_results= validate_model(session, val_data, x,y,e,phase_train,prob) 
    remain_results = np.array([correct_results[i] for i in remain] )
    labels = [val_data[1][i] for i in remain] 
    acc,correct_l,correct_g,correct_t= each_perform(remain_results,labels )  
session.close()
reset() 
id_True =  np.nonzero(remain_results==True) 
overall_acc = (len(id_True[0])/len(remain_results)) 
print ('LT:', 100*np.mean( correct_l))
print ('GT:', 100*np.mean( correct_g))
print ('TP:', 100*np.mean( correct_t))
print ('Overall:', 100*overall_acc) 
    

Loading model from file 'E:/05_research/04_deep_learning_model/Save_parameters/modelA/Best_noise_model/modelA_snr50_fully'...
INFO:tensorflow:Restoring parameters from E:/05_research/04_deep_learning_model/Save_parameters/modelA/Best_noise_model/modelA_snr50_fully
Loading validation data...
Validating model...
LT: 81.44329896907216
GT: 82.17821782178217
TP: 97.2972972972973
Overall: 87.37864077669903
